In [1]:
# Python program to read
# json file

from numpy import power, linspace, sqrt, log, exp, abs, max, conj
from numpy.fft import fft, fftshift, fftfreq, ifft
from scipy.constants import c
#import pypret
from matplotlib import pyplot as plt

import numpy as np
#import pylab as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d,  UnivariateSpline
from scipy.optimize import OptimizeResult

import pandas as pd

import json

In [10]:

#%%

# calculation of FWHM of pulse temporal profile
def FWHM(X, Y):
    
    half_max = max(Y)/2.0
    
    spline01 = UnivariateSpline(X, Y-half_max, s=0)
    
    R = []

    for r in spline01.roots():
    
        R.append(r)
        
    return (round(abs(R[0]-R[-1])))

# calculates temporal profile of a pulse from spectrum and spectral phase
def temporal(n_points, step, spectrum_freq_abs, spec_phase):

    time = fftshift(fftfreq(n_points, d=abs(step)))

    spectrum_freq_abs = np.pad(spectrum_freq_abs, (10000, 10000), 'constant', constant_values=(0, 0))
    spec_phase = np.pad(spec_phase, (10000, 10000), 'constant', constant_values=(0, 0))


    field = fftshift(fft(spectrum_freq_abs*exp(1j*spec_phase)))
    field_TL = fftshift(fft(spectrum_freq_abs))


    X = time*1e15
    Y = ((field*conj(field)))

    tnew = np.linspace(-200, 200, 10000, endpoint=True)


    Y = Y/max(Y)
    
    return (Y)


# spectral phase polynomial
def fmodel(x, w):
    # return (w[0] + w[1]*(x/1e12) + w[2] * (x/1e12)**2 + w[3] * (x/1e12)**3 + w[4] * (x/1e12)**4) #  + w[5] * (x/1e12)**5 + w[6] * (x/1e12)**6)
    return (w[0] * (x/1e12)**2 + w[1] * (x/1e12)**3 + w[2] * (x/1e12)**4) #  + w[5] * (x/1e12)**5 + w[6] * (x/1e12)**6)

def Phase(x, w):
    
    return (1/2)*(w[0])*1e-24*(2*np.pi*xnew - w0)**2 + (1/6)*(w[1])*1e-36*(2*np.pi*xnew - w0)**3 + (1/24)*(w[2])*1e-48*(2*np.pi*xnew - w0)**4 
    
     
    
# rmse
def rmse(ww1):
    
    xnew = np.linspace(290.3, 291.4, n_points, endpoint=True)*10**12
    
    step = (xnew[0] - xnew[1])
    
    phase = Phase(xnew, ww1)
    
    Y_TL = np.real(temporal(n_points, step, spectrum_freq_abs, 0*phase))
    
    Y = np.real(temporal(n_points, step, spectrum_freq_abs, phase))    
    
    i1 = np.argmax(Y_TL)
    i2 = np.argmax(Y)

    delta = (i1 - i2)

    Y = np.roll(Y, delta)
    
#     return ((sum((Y-Y_TL)**2) / len(Y)))
    return ((sum((Y-Y_TL))))

def rmse1(Y_TL, Y):
    
    # step = (xnew[0] - xnew[1])
    
    i1 = np.argmax(Y_TL)
    i2 = np.argmax(Y)

    delta = (i1 - i2)

    Y = np.roll(Y, delta)
    
    # I = np.real(temporal(n_points, step, spectrum_freq_abs, fmodel(xnew, ww1)))  
    
    err = (sum((Y-Y_TL)**2) / len(Y))
    
    return err

def rounded_nelder_mead(fun, x0, bounds, print_progress, decimal_precision1=4, decimal_precision2=4, decimal_precision3=4, maxiter=100, alpha=1.3487640718112321, gamma=2.139744408219208, rho=0.3740185211921033, sigma=0.5501390442296267, tol=1e-4):
    n = len(x0)

    simplex = np.zeros((n + 1, n))
    simplex[0] = x0

    for i in range(n):
        point = np.array(x0)
        point[i] = bounds[i][0]
        simplex[i + 1] = point

        point = np.array(x0)
        point[i] = bounds[i][1]
        simplex[i + 1] = point

    fs = np.zeros(n + 1)
    for i in range(n + 1):
        fs[i] = fun(simplex[i])

    nfev = n + 1  # Initialize the number of function evaluations

    optimized_solutions = []  # List to store optimized solutions

    for i in range(maxiter):
        # Sort simplex according to function values
        idx = np.argsort(fs)
        simplex = simplex[idx]
        fs = fs[idx]

        # Calculate the centroid of the simplex
        xbar = np.mean(simplex[:-1], axis=0)

        # Reflection
        xr = xbar + alpha * (xbar - simplex[-1])
        xr = np.clip(xr, bounds[:, 0], bounds[:, 1])
        xr[0] = np.round(xr[0], decimals=decimal_precision1)
        xr[1] = np.round(xr[1], decimals=decimal_precision2)
        xr[2] = np.round(xr[2], decimals=decimal_precision3)
        fxr = fun(xr)
        nfev += 1  # Increment the number of function evaluations

        if fs[0] <= fxr < fs[-2]:
            # Successful reflection
            simplex[-1] = xr
            fs[-1] = fxr

        elif fxr < fs[0]:
            # Expansion
            xe = xbar + gamma * (xr - xbar)
            xe = np.clip(xe, bounds[:, 0], bounds[:, 1])
            xe[0] = np.round(xe[0], decimals=decimal_precision1)
            xe[1] = np.round(xe[1], decimals=decimal_precision2)
            xe[2] = np.round(xe[2], decimals=decimal_precision3)
            fxe = fun(xe)
            nfev += 1  # Increment the number of function evaluations

            if fxe < fxr:
                simplex[-1] = xe
                fs[-1] = fxe
            else:
                simplex[-1] = xr
                fs[-1] = fxr

        else:
            # Contraction
            xc = xbar + rho * (simplex[-1] - xbar)
            xc = np.clip(xc, bounds[:, 0], bounds[:, 1])
            xc[0] = np.round(xc[0], decimals=decimal_precision1)
            xc[1] = np.round(xc[1], decimals=decimal_precision2)
            xc[2] = np.round(xc[2], decimals=decimal_precision3)
            
            fxc = fun(xc)
            nfev += 1  # Increment the number of function evaluations

            if fxc < fs[-1]:
                simplex[-1] = xc
                fs[-1] = fxc

            else:
                # Shrink
                simplex[1:] = simplex[0] + sigma * (simplex[1:] - simplex[0])
                for j in range(1, n + 1):
                    simplex[j][0] = np.round(simplex[j][0], decimals=decimal_precision1)
                    simplex[j][1] = np.round(simplex[j][1], decimals=decimal_precision2)
                    simplex[j][2] = np.round(simplex[j][2], decimals=decimal_precision3)
                    fs[j] = fun(simplex[j])
                    nfev += 1  # Increment the number of function evaluations
        
        if print_progress == 1:
            print('iteration:', i+1)
            print('best value of objective function so far:', fs[0])
            print('best candidate:', simplex[0])
            
        # Round and append optimized solution to the list with individual decimal precision for each parameter
        rounded_solution = (
            np.round(simplex[0][0], decimals=decimal_precision1),
            np.round(simplex[0][1], decimals=decimal_precision2),
            np.round(simplex[0][2], decimals=decimal_precision3)
        )
        optimized_solutions.append(rounded_solution)
                
        # Check termination criteria
        if np.max(np.abs(fs[:-1] - fs[-1])) < tol:
            break

    # Round optimized solution with individual decimal precision for each parameter
    optimized_solution = (
        np.round(simplex[0][0], decimals=decimal_precision1),
        np.round(simplex[0][1], decimals=decimal_precision2),
        np.round(simplex[0][2], decimals=decimal_precision3)
    )
    min_function_value = fs[0]
    success = True if i < maxiter - 1 else False
    termination_reason = 'Maximum number of iterations exceeded.' if i == maxiter - 1 else 'Maximum number of iterations not exceeded'
    num_iterations = i + 1

    return optimized_solution, min_function_value, nfev, success, termination_reason, num_iterations, optimized_solutions

# def de(rmse, bounds, mut=0.8, crossp=0.7, popsize=30):
def de(rmse, bounds, mut=0.8, crossp=0.8, popsize=20, its=1000):
# def de(rmse, bounds, mut=0.8, crossp=0.8, popsize=20, its=1000):
    dimensions = len(bounds)
    print('(0)', dimensions)
    pop = np.random.rand(popsize, dimensions)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    print('(1)', min_b)
    print('(2)', pop)
    print('(3)', diff)
    pop_denorm = min_b + pop * diff
    fitness = np.asarray([rmse(ind) for ind in pop_denorm])
    best_idx = np.argmin(fitness)
    best = pop_denorm[best_idx]
    # for i in range(its):
    i = 0
    while fitness[best_idx] > 0.000001:
        # i += 1
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace = False)]
            mutant = np.clip(a + mut * (b - c ), 0, 1)
            cross_points = np.random.rand(dimensions) < crossp
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])
            trial_denorm = min_b + trial * diff
            f = rmse(trial_denorm)
            i += 1
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial_denorm
        # yield best, fitness[best_idx]
    yield min_b + pop * diff, fitness[best_idx], best_idx

    print (" i =", i )


In [11]:
with open('C:/Users/isrqa/Downloads/9_11_2023Data_StretcherFrog2-31 PM.json') as user_file:
  F = json.load(user_file)

# C:\Users\isrqa\Downloads
keysList = list(F[0].keys())

#  ['Spec_Orders', 'Stretcher', 'SpectrumX_Values', 'SpectrumY_Values']

X = np.array(F[0]['SpectrumX_Values'])*1e-9
Y = np.array(F[0]['SpectrumY_Values'])


X = np.divide(3e8, X)

#%%

n_points = 1000

# x = np.sqrt(X)

Y = np.array(Y)



y = np.sqrt(Y)


Y =np.array(Y)

y = np.array(y)

f = interp1d(X, Y)
f2 = interp1d(X, y, kind='cubic')
# p2 = interp1d(x, phase, kind='cubic')


xnew = np.linspace(290.3, 291.4, n_points, endpoint=True)*10**12

spectrum_freq_abs01 = f(xnew)
spectrum_freq_abs = f2(xnew)
# spectrum_freq_abs_phase = f2(xnew)
# spec_phase = p2(xnew)

#%%

half_max = max(spectrum_freq_abs01)/2.0

spline00 = UnivariateSpline(xnew, spectrum_freq_abs01-half_max, s=0)
r01, r02, r03, r04, r05, r06 = 3e17/spline00.roots()
print(r01, r06)

l0 = abs(r01+r06)/2

w0 = 2*np.pi*3e8/(l0*1e-9)


#%%

GDD = 4.6

TOD = 5

FOD = -5

W = [GDD, TOD, FOD]

# phase = (1/2)*(GDD*1000 - 25000)*1e-30*(2*np.pi*xnew - w0)**2 + (1/6)*(TOD*1000-30000)*1e-45*(2*np.pi*xnew - w0)**3 + (1/24)*(FOD*1000 - 50000)*1e-60*(2*np.pi*xnew - w0)**4 
# phase = (1/2)*(GDD)*1e-24*(2*np.pi*xnew - w0)**2 + (1/6)*(TOD)*1e-36*(2*np.pi*xnew - w0)**3 + (1/24)*(FOD)*1e-48*(2*np.pi*xnew - w0)**4 

# plt.plot(3e17/xnew, phase)

phase = Phase(xnew, W)

#%%

n = 4

# z = np.polyfit(xnew, spec_phase, n)
zz = np.polyfit(xnew/10**12, phase, n)
    
w = []
w1 = []
ww = []
ww1 = []

# for i in range(n+1)[::-1]:
    
#     w.append(z[i]/((1/2)*1e-30*(2*np.pi)**i))
#     w1.append(z[i])
    

for i in range(n+1)[::-1]:
    
    ww.append(zz[i]/((1/2)*(1e-12*2*np.pi)**i))
    ww1.append(zz[i])
    


#%%

phase = 0

for i in range(n+1):
    
    phase += ww1[i]*(xnew)**i/(1e12)**i
    # phase += w1[i]*(xnew)**i
    


#%%


# plt.figure('Spectral')

fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Spectrl Power Density (a.u.)', color=color)
ax1.plot(X, Y, color=color)
# ax1.plot(3e17/x, yy, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Spectral phase (rad)', color=color)  # we already handled the x-label with ax1
ax2.plot(xnew, phase, color=color)
# ax2.plot(3e17/xnew, phase, color=color)
# ax2.plot(xnew, phase, color=color)
# ax2.plot(3e17/xnew, fmodel(xnew, ww1), 'b')
# ax2.plot(3e17/xnew, fmodel(xnew, ww1), 'g')
ax2.tick_params(axis='y', labelcolor=color)
plt.ylim(-10, 10)
# plt.figure('Spectral')
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()



#%%
# FFT
step = (xnew[0] - xnew[1])
n_points += 20000

time = fftshift(fftfreq(n_points, d=abs(step)))

X = time*1e12

#%%

Y = np.real(temporal(n_points, step, spectrum_freq_abs, phase )) 

Y_TL = np.real(temporal(n_points, step, spectrum_freq_abs, 0*phase))


# print("delta_t TL = ", FWHM(X, Y_TL))    
# print("delta_t = ", FWHM(X, Y))

# i1 = np.argmax(Y_TL)
# i2 = np.argmax(Y)

# delta = (i1 - i2)

# Y = np.roll(Y, delta)

#%%


plt.figure('Temporal')
plt.xlabel('Time (ps)')
plt.ylabel('Intensity (a.u.)')

plt.plot(X[10300:10690], abs(Y_TL[10300:10690]),  'r', label='FWHM(TL) = 28 fs')
plt.plot(X[10300:10690], abs(Y[10300:10690]), 'b', label='FWHM = 67 fs')

# plt.plot(X, Y_TL)
# plt.plot(X, Y)

# plt.legend()
plt.show()

#%%


print('rmse =',rmse1(Y_TL, Y))


#%%


# bounds = [(-1.e+04, 1.e+04),
#  (-1.e+03, 1.e+03),
#  (-1.e+02, 1.e+02),
#  (-1.e-01, 1.e-01),
#  (-1.e-05, 1.e-05)] 

print_progress = 1
num_dimensions = 3

# Define the bounds
lower_bound = -5.0
upper_bound = 5.0

# # Generate a random array within the bounds for each dimension
x0 = np.random.uniform(low=lower_bound, high=upper_bound, size=(num_dimensions,))

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# R = bounded_nelder_mead(rmse, x0, np.array([[-5.0, 5.0], [-5.0, 5.0], [-5.0, 5.0]]), 1)
optimized_solution, min_function_value, nfev, success, termination_reason, num_iterations, optimized_solutions = rounded_nelder_mead(rmse, x0, np.array([[-5.0, 5.0], [-5.0, 5.0], [-5.0, 5.0]]), 1, decimal_precision1 = 3, decimal_precision2 = 4, decimal_precision3 = 5)

# # Number of dimensions
# result = list(de(rmse, bounds))

# # Run the optimization using Nelder-Mead method
# # optimized_solution, min_function_value, nfev, success, termination_reason, num_iterations, optimized_solutions = nelder_mead_adaptive(rmse, np.array([1, 4.2, -1.1]), np.array([[-5.0, 5.0], [-5.0, 5.0], [-5.0, 5.0]]), print_progress)
# optimized_solution, min_function_value, nfev, success, termination_reason, num_iterations, optimized_solutions = adaptive_nelder_mead(rmse, x0, np.array([[-5.0, 5.0], [-5.0, 5.0], [-5.0, 5.0]]), print_progress, maxiter=300, alpha = 0.8296584594198916, gamma = 0.5888754856895072, rho = 1.6758636476717481, sigma = 1.4991271372932704, tol=1e-4)

# ([0.8296584594198916, 0.5888754856895072, 1.6758636476717481, 1.4991271372932704])

# phase = Phase(xnew, optimized_solution)

# print (R)
# phase = Phase(xnew, R.x)
phase = Phase(xnew, optimized_solution)

#%%

n = 4

# z = np.polyfit(xnew, spec_phase, n)
zz = np.polyfit(xnew/10**12, phase, n)
    
w = []
w1 = []
ww = []
ww1 = []

# for i in range(n+1)[::-1]:
    
#     w.append(z[i]/((1/2)*1e-30*(2*np.pi)**i))
#     w1.append(z[i])
    

for i in range(n+1)[::-1]:
    
    ww.append(zz[i]/((1/2)*(1e-12*2*np.pi)**i))
    ww1.append(zz[i])
    


#%%

phase = 0

for i in range(n+1):
    
    phase += ww1[i]*(xnew)**i/(1e12)**i
    # phase += w1[i]*(xnew)**i
    


#%%


# plt.figure('Spectral')

# fig, ax1 = plt.subplots()

# color = 'tab:blue'
# ax1.set_xlabel('Frequency (Hz)')
# ax1.set_ylabel('Spectrl Power Density (a.u.)', color=color)
# ax1.plot(X, Y, color=color)
# # ax1.plot(3e17/x, yy, color=color)
# ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:red'
# ax2.set_ylabel('Spectral phase (rad)', color=color)  # we already handled the x-label with ax1
# ax2.plot(xnew, phase, color=color)
# # ax2.plot(3e17/xnew, phase, color=color)
# # ax2.plot(xnew, phase, color=color)
# # ax2.plot(3e17/xnew, fmodel(xnew, ww1), 'b')
# # ax2.plot(3e17/xnew, fmodel(xnew, ww1), 'g')
# ax2.tick_params(axis='y', labelcolor=color)
# plt.ylim(-10, 10)
# # plt.figure('Spectral')
# fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()



#%%
# FFT
step = (xnew[0] - xnew[1])
n_points = 21000

time = fftshift(fftfreq(n_points, d=abs(step)))

X = time*1e12

#%%

Y = np.real(temporal(n_points, step, spectrum_freq_abs, phase )) 

Y_TL = np.real(temporal(n_points, step, spectrum_freq_abs, 0*phase))


# print("delta_t TL = ", FWHM(X, Y_TL))    
# print("delta_t = ", FWHM(X, Y))

# i1 = np.argmax(Y_TL)
# i2 = np.argmax(Y)

# delta = (i1 - i2)

# Y = np.roll(Y, delta)

#%%


plt.figure('Temporal')
plt.xlabel('Time (ps)')
plt.ylabel('Intensity (a.u.)')

plt.plot(X[10300:10690], abs(Y_TL[10300:10690]),  'r', label='FWHM(TL) = 28 fs')
plt.plot(X[10300:10690], abs(Y[10300:10690]), 'b', label='FWHM = 67 fs')

# plt.plot(X, Y_TL)
# plt.plot(X, Y)

# plt.legend()
plt.show()

#%%


# print('rmse =',rmse1(Y_TL, Y))

NameError: name 'json' is not defined